In [1]:
#standard libraries
import csv
import os
import time
from collections import namedtuple

#custom libraries
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
def read_file_format(filename_queue):
    reader = tf.TextLineReader(skip_header_lines=1)
    _, value = reader.read(filename_queue)

    record_defaults = [tf.constant([], dtype=tf.int32), tf.constant([], dtype=tf.int32)]
    _, col2 = tf.decode_csv(value, record_defaults=record_defaults)
    
    example = tf.stack([col2])
    return example

In [3]:
def input_pipeline(filenames, batch_size = 3, seq_length=3,
                   num_epochs = None, evaluation = False):   
    filename_queue = tf.train.string_input_producer(
        filenames, num_epochs=num_epochs, shuffle=False)

    example = read_file_format(filename_queue)
        
    min_after_dequeue = 10
    capacity = min_after_dequeue + 3 * batch_size
    example_batch = tf.train.batch(
        [example], batch_size=batch_size*seq_length, capacity=capacity
    )    

    label_batch = tf.concat(
        [example_batch[-1], example_batch[1:,0]],
        axis=0)

    example_batch = tf.reshape(example_batch, (batch_size, seq_length))
    label_batch = tf.reshape(label_batch, (batch_size, seq_length))

    return example_batch, label_batch

In [4]:
def _activation_summary(x):
    tensor_name = x.name
    tensor_name = tensor_name.replace(':', '_')
    tensor_name = tensor_name.replace('(', '_')
    tensor_name = tensor_name.replace(')', '_')
    tensor_name = tensor_name.replace(' ', '_')

    tf.summary.histogram(tensor_name + '/activations', x)
    tf.summary.scalar(tensor_name + '/sparsity', tf.nn.zero_fraction(x))

In [5]:
def _add_loss_summaries(total_loss, averager, include_averaged_loss=False):
    # Compute the moving average of all individual losses and the total loss.
    losses = tf.get_collection('losses')
    if include_averaged_loss:
        loss_averages_op = averager.apply(losses + [total_loss])

    # Attach a scalar summary to all individual losses and the total loss; do the
    # same for the averaged version of the losses.
    for l in losses + [total_loss]:
    # Name each loss as '(raw)' and name the moving average version of the loss
    # as the original loss name.
        l_name = l.name.replace(":", "_")

        tf.summary.scalar(l_name + '_raw_', l)        
        if include_averaged_loss:
            tf.summary.scalar(l_name + '_raw_', l)
            tf.summary.scalar(l_name, averager.average(l))
        
    if include_averaged_loss:
        return loss_averages_op
    else:
        return total_loss

In [6]:
class Model():
    
    def __init__(self, args, infer=False):
        if infer:
            self.batch_size = 1
            self.seq_length = 1
        else:
            self.batch_size = args.batch_size
            self.seq_length = args.seq_length

        self.x = tf.placeholder(tf.int32, shape=[args.batch_size, args.seq_length])
        self.y_ = tf.placeholder(tf.int32, shape=[args.batch_size, args.seq_length])

        self.lstm_cell = tf.contrib.rnn.BasicLSTMCell(args.rnn_size)
        self.lstm = tf.contrib.rnn.MultiRNNCell([self.lstm_cell]*args.num_layers)

        self.initial_state = self.lstm.zero_state(args.batch_size, tf.float32)

        with tf.variable_scope('lstm'):
            W = tf.get_variable('W', [args.rnn_size, args.vocab_size], tf.float32, tf.random_normal_initializer())
            b = tf.get_variable('b', [args.vocab_size], tf.float32, tf.constant_initializer(0.0))

            # Define Embedding
            embedding_mat = tf.get_variable('embedding', [args.vocab_size, args.rnn_size],
                                            tf.float32, tf.random_normal_initializer())
            embedding_output = tf.nn.embedding_lookup(embedding_mat, self.x)

            rnn_inputs = tf.split(axis=1, num_or_size_splits=self.seq_length, value=embedding_output)
            rnn_inputs = [tf.squeeze(x, [1]) for x in rnn_inputs]

            def loop(prev, _):
                prev = tf.matmul(prev, W) + b
                prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
                return tf.nn.embedding_lookup(embedding, prev_symbol)

        outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(rnn_inputs,
                                                                    self.initial_state,
                                                                    self.lstm,
                                                                    loop_function=loop if infer else None,
                                                                    scope='lstm')
        output = tf.reshape(tf.concat(outputs,1), [-1, args.rnn_size])

        self.logits = tf.matmul(output, W) + b
        self.softmax_p = tf.nn.softmax(self.logits)

        self.loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [self.softmax_p],
            [self.y_],
            [tf.ones([args.batch_size * args.seq_length], dtype=tf.float32)]
        )
        tf.add_to_collection('losses', self.loss)
        tf.add_n(tf.get_collection('losses'), name='total_loss')
    
#            # Generate moving averages of all losses and associated summaries.
#         loss_avgs = tf.train.ExponentialMovingAverage(0.9, name='avg')
#         losses = _add_loss_summaries(self.loss, loss_avgs, True)

        # Compute gradients.
#         with tf.control_dependencies([losses]):
        opt = tf.train.AdamOptimizer(args.learning_rate)
        grads = opt.compute_gradients(self.loss)
        trunc_grads = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in grads]


        # Apply gradients.
        apply_gradient_op = opt.apply_gradients(trunc_grads, global_step=global_step)

        # Add histograms for trainable variables.
        for var in tf.trainable_variables():
            tf.summary.histogram(var.op.name, var)

        # Add histograms for gradients.
        for grad, var in trunc_grads:
            if grad is not None:
                tf.summary.histogram(var.op.name + '/gradients', grad)


        if args.compute_variable_averages:
            variable_averages = tf.train.ExponentialMovingAverage(0.9, name='avg')
            variables_averages_op = variable_averages.apply(tf.trainable_variables())
            
            with tf.control_dependencies([apply_gradient_op, variables_averages_op]):
                self.train_op = tf.no_op(name='train')
        else:
            with tf.control_dependencies([apply_gradient_op]):
                self.train_op = tf.no_op(name='train')
        
        
    def sample(self):
        flat_y_ = tf.expand_dims(tf.reshape(self.y_, [-1]), 1)
        flat_y_ = tf.cast(flat_y_, tf.int64)
        samples = tf.multinomial(self.softmax_p, 1)
        
        self.sampled_results = tf.concat([flat_y_, samples], axis=1)
        return self.sampled_results

In [7]:
data_path = "data"

vocab_file = "vocab1.csv"

train_file = "train1.csv"

model_path = 'VanillaLSTM'

In [8]:
# Download/store Shakespeare data
full_model_dir = os.path.join(data_path, model_path)

# Make Model Directory
if not os.path.exists(full_model_dir):
    os.makedirs(full_model_dir)

In [9]:
vocab = pd.read_csv("{0}/{1}".format(data_path, vocab_file),
                    header=None)

In [10]:
class ArgStruct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

In [11]:
arg_dict = {
    'data_path': data_path,
    'model_path': model_path,
    'rnn_size': 256,
    'num_layers': 7,
    'batch_size': 16,
    'seq_length': 16,
    'num_epochs': 1,
    'learning_rate': 0.0001,
    'momentum': 0.9,
    'logdir': 'TF_Logs',
    'vocab_size': len(vocab)+1,
    'save_every': 100,
    'print_every': 10,
    'compute_variable_averages': True
}

In [ ]:
args = ArgStruct(**arg_dict)

In [ ]:
with tf.Graph().as_default():

    global_step = tf.Variable(0, name='global_step', trainable=False)

    model = Model(args=args, infer=False)

    example_feed, label_feed = input_pipeline(
        ["{0}/{1}".format(args.data_path, train_file)],
        batch_size=args.batch_size,
        seq_length=args.seq_length,
        num_epochs=args.num_epochs)
    
    with tf.Session() as sess: 
        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter(args.logdir, sess.graph)
        
        #initialize all variables
        init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        sess.run(init_op)
        
        # Start populating the filename queue.
        coord = tf.train.Coordinator()  
        threads = tf.train.start_queue_runners(coord=coord, sess=sess)
        
        saver = tf.train.Saver(tf.global_variables())
        
        while not coord.should_stop():
            try:
                start_time = time.time()                

                example_batch, label_batch = sess.run([example_feed,
                                                      label_feed])
                
                if global_step.eval() % args.print_every == 0:
                    latest_loss = sess.run([model.loss],
                                           feed_dict={model.x: example_batch,
                                                      model.y_: label_batch})
                    try:
                        summary_nums = (global_step.eval(), duration,
                                        np.mean(latest_loss))
                        print('Iteration: {0}, Last Step Duration: {1}, Loss: {2}'.format(*summary_nums))
                    except:
                        pass
                    sampled_results = model.sample()
                    results = sess.run([model.sampled_results],
                                       feed_dict={model.x: example_batch,
                                                  model.y_: label_batch})
                    results = pd.DataFrame(results[0]).T
                    recode_results = results.replace(vocab.set_index(1).to_dict().get(0))
                    if not os.path.exists("translation.txt"):
                        recode_results.to_csv("translation.txt", header=False, index=False, sep="\t", mode="w")
                    else:
                        recode_results.to_csv("translation.txt", header=False, index=False, sep="\t", mode="a")

                result, summary =  sess.run(
                    [model.train_op, merged],
                    feed_dict={model.x: example_batch,
                               model.y_: label_batch})
                writer.add_summary(summary, global_step.eval())
                
                duration = time.time() - start_time
        
                # Save the model and the vocab
                if global_step.eval() % args.save_every == 0:
                    # Save model
                    model_file_name = os.path.join(full_model_dir, 'model')
                    saver.save(sess, model_file_name, global_step=global_step)
                    print('Model Saved To: {}'.format(model_file_name))

            except (tf.errors.OutOfRangeError, tf.errors.InvalidArgumentError) as e:
           
                print('Done training for %d epochs, %d steps.' % (args.num_epochs, global_step.eval()))
                # When done, ask the threads to stop.
                coord.request_stop()

        
        # Wait for threads to finish.
        coord.join(threads)
        sess.close()

Iteration: 10, Last Step Duration: 0.845599889755249, Loss: 3.576180934906006
Iteration: 20, Last Step Duration: 0.7775516510009766, Loss: 3.534910202026367
Iteration: 30, Last Step Duration: 0.7775511741638184, Loss: 3.5098087787628174
Iteration: 40, Last Step Duration: 3.185258626937866, Loss: 3.493947982788086
Iteration: 50, Last Step Duration: 1.4710440635681152, Loss: 3.488189935684204
Iteration: 60, Last Step Duration: 1.1137900352478027, Loss: 3.4876821041107178
Iteration: 70, Last Step Duration: 1.0867717266082764, Loss: 3.456963062286377
Iteration: 80, Last Step Duration: 0.9756934642791748, Loss: 3.47416615486145
Iteration: 90, Last Step Duration: 0.8125760555267334, Loss: 3.481137752532959
Model Saved To: data\VanillaLSTM\model
Iteration: 100, Last Step Duration: 0.8716189861297607, Loss: 3.4552900791168213
Iteration: 110, Last Step Duration: 0.8215816020965576, Loss: 3.4734292030334473
Iteration: 120, Last Step Duration: 0.9486725330352783, Loss: 3.4412741661071777
Iteratio

Iteration: 990, Last Step Duration: 0.7565364837646484, Loss: 3.2346994876861572
Model Saved To: data\VanillaLSTM\model
Iteration: 1000, Last Step Duration: 0.7455296516418457, Loss: 3.2882485389709473
Iteration: 1010, Last Step Duration: 0.7695462703704834, Loss: 3.2756543159484863
Iteration: 1020, Last Step Duration: 0.7495317459106445, Loss: 3.295344114303589
Iteration: 1030, Last Step Duration: 0.7475311756134033, Loss: 3.251082420349121
Iteration: 1040, Last Step Duration: 0.7535336017608643, Loss: 3.2622151374816895
Iteration: 1050, Last Step Duration: 0.777552604675293, Loss: 3.271091938018799
Iteration: 1060, Last Step Duration: 0.7275164127349854, Loss: 3.2751002311706543
Iteration: 1070, Last Step Duration: 0.7545349597930908, Loss: 3.2837138175964355
Iteration: 1080, Last Step Duration: 0.7480218410491943, Loss: 3.2509078979492188
Iteration: 1090, Last Step Duration: 0.7355198860168457, Loss: 3.2510650157928467
Model Saved To: data\VanillaLSTM\model
Iteration: 1100, Last Ste

Iteration: 1950, Last Step Duration: 0.7495317459106445, Loss: 3.209127902984619
Iteration: 1960, Last Step Duration: 0.7245128154754639, Loss: 3.1997737884521484
Iteration: 1970, Last Step Duration: 0.7595388889312744, Loss: 3.1888089179992676
Iteration: 1980, Last Step Duration: 0.7325184345245361, Loss: 3.179514169692993
Iteration: 1990, Last Step Duration: 0.752532958984375, Loss: 3.186537742614746
Model Saved To: data\VanillaLSTM\model
Iteration: 2000, Last Step Duration: 0.7485289573669434, Loss: 3.192274332046509
Iteration: 2010, Last Step Duration: 0.7255139350891113, Loss: 3.1542210578918457
Iteration: 2020, Last Step Duration: 0.7445278167724609, Loss: 3.186152935028076
Iteration: 2030, Last Step Duration: 0.7355213165283203, Loss: 3.1566286087036133
Iteration: 2040, Last Step Duration: 0.7445285320281982, Loss: 3.170045852661133
Iteration: 2050, Last Step Duration: 0.7345192432403564, Loss: 3.201216459274292
Iteration: 2060, Last Step Duration: 0.7395257949829102, Loss: 3.16

Iteration: 2910, Last Step Duration: 0.8295879364013672, Loss: 3.098173141479492
Iteration: 2920, Last Step Duration: 0.7405247688293457, Loss: 3.100174903869629
Iteration: 2930, Last Step Duration: 0.7365221977233887, Loss: 3.1120378971099854
Iteration: 2940, Last Step Duration: 0.729517936706543, Loss: 3.1369431018829346
Iteration: 2950, Last Step Duration: 0.7595391273498535, Loss: 3.1324963569641113
Iteration: 2960, Last Step Duration: 0.8836262226104736, Loss: 3.098220109939575
Iteration: 2970, Last Step Duration: 0.9256565570831299, Loss: 3.1409835815429688
Iteration: 2980, Last Step Duration: 0.7385244369506836, Loss: 3.1219820976257324
Iteration: 2990, Last Step Duration: 0.727515697479248, Loss: 3.1066291332244873
Model Saved To: data\VanillaLSTM\model
Iteration: 3000, Last Step Duration: 0.7385239601135254, Loss: 3.1058499813079834
Iteration: 3010, Last Step Duration: 0.7465288639068604, Loss: 3.1025896072387695
Iteration: 3020, Last Step Duration: 0.7415256500244141, Loss: 3

Iteration: 3880, Last Step Duration: 0.7205121517181396, Loss: 3.1134071350097656
Iteration: 3890, Last Step Duration: 0.8025689125061035, Loss: 3.144108295440674
Model Saved To: data\VanillaLSTM\model
Iteration: 3900, Last Step Duration: 0.7405242919921875, Loss: 3.1020541191101074
Iteration: 3910, Last Step Duration: 0.7565360069274902, Loss: 3.082491397857666
Iteration: 3920, Last Step Duration: 0.8045706748962402, Loss: 3.1362955570220947
Iteration: 3930, Last Step Duration: 0.7455294132232666, Loss: 3.086503028869629
Iteration: 3940, Last Step Duration: 0.7475295066833496, Loss: 3.105705738067627
Iteration: 3950, Last Step Duration: 0.7365219593048096, Loss: 3.10227632522583
Iteration: 3960, Last Step Duration: 0.7595393657684326, Loss: 3.0868113040924072
Iteration: 3970, Last Step Duration: 0.7945632934570312, Loss: 3.1172609329223633
Iteration: 3980, Last Step Duration: 0.7645418643951416, Loss: 3.13602614402771
Iteration: 3990, Last Step Duration: 0.7765512466430664, Loss: 3.14

...

Iteration: 19270, Last Step Duration: 0.7595233917236328, Loss: 3.028092622756958
Iteration: 19280, Last Step Duration: 0.7605257034301758, Loss: 3.0242080688476562
Iteration: 19290, Last Step Duration: 0.7585372924804688, Loss: 3.0512161254882812
Model Saved To: data\VanillaLSTM\model
Iteration: 19300, Last Step Duration: 0.7754781246185303, Loss: 3.0589959621429443
Iteration: 19310, Last Step Duration: 0.7585361003875732, Loss: 3.0396289825439453
Iteration: 19320, Last Step Duration: 0.7383666038513184, Loss: 3.051271438598633
Iteration: 19330, Last Step Duration: 0.7595386505126953, Loss: 3.1091599464416504
Iteration: 19340, Last Step Duration: 0.7605400085449219, Loss: 3.058915376663208
Iteration: 19350, Last Step Duration: 0.7715599536895752, Loss: 3.0511889457702637
Iteration: 19360, Last Step Duration: 0.7660665512084961, Loss: 3.05509090423584
Iteration: 19370, Last Step Duration: 0.781552791595459, Loss: 3.0203299522399902
Iteration: 19380, Last Step Duration: 0.76359772682189

Iteration: 20220, Last Step Duration: 0.8040914535522461, Loss: 3.051217555999756
Iteration: 20230, Last Step Duration: 0.8205821514129639, Loss: 3.0434656143188477
Iteration: 20240, Last Step Duration: 0.8185806274414062, Loss: 2.9855196475982666
Iteration: 20250, Last Step Duration: 0.8145627975463867, Loss: 3.0396676063537598
Iteration: 20260, Last Step Duration: 0.8726048469543457, Loss: 3.0280041694641113
Iteration: 20270, Last Step Duration: 0.8775579929351807, Loss: 3.0434560775756836
Iteration: 20280, Last Step Duration: 0.9066464900970459, Loss: 3.062758445739746
Iteration: 20290, Last Step Duration: 0.8526027202606201, Loss: 3.055081367492676
Model Saved To: data\VanillaLSTM\model
Iteration: 20300, Last Step Duration: 0.880486011505127, Loss: 3.062798500061035
Iteration: 20310, Last Step Duration: 0.7800769805908203, Loss: 3.0822038650512695
Iteration: 20320, Last Step Duration: 0.7995650768280029, Loss: 3.0396666526794434
Iteration: 20330, Last Step Duration: 0.8369357585906

Iteration: 21180, Last Step Duration: 0.9108855724334717, Loss: 3.0859944820404053
Iteration: 21190, Last Step Duration: 0.9897022247314453, Loss: 3.016475200653076
Model Saved To: data\VanillaLSTM\model
Iteration: 21200, Last Step Duration: 0.9016389846801758, Loss: 3.096014976501465
Iteration: 21210, Last Step Duration: 0.8977963924407959, Loss: 3.020313262939453
Iteration: 21220, Last Step Duration: 1.0517451763153076, Loss: 3.0241916179656982
Iteration: 21230, Last Step Duration: 1.0877718925476074, Loss: 3.06280517578125
Iteration: 21240, Last Step Duration: 0.8649814128875732, Loss: 2.993309497833252
Iteration: 21250, Last Step Duration: 0.8637487888336182, Loss: 2.9971847534179688
Iteration: 21260, Last Step Duration: 0.9368870258331299, Loss: 3.058955669403076
Iteration: 21270, Last Step Duration: 0.8828599452972412, Loss: 3.055068254470825
Iteration: 21280, Last Step Duration: 0.8816113471984863, Loss: 3.074392318725586
Iteration: 21290, Last Step Duration: 0.8886163234710693,

Iteration: 22130, Last Step Duration: 0.8467047214508057, Loss: 3.0182418823242188
Iteration: 22140, Last Step Duration: 0.7960069179534912, Loss: 3.0164780616760254
Iteration: 22150, Last Step Duration: 0.782555103302002, Loss: 3.08217191696167
Iteration: 22160, Last Step Duration: 0.7887701988220215, Loss: 3.0705971717834473
Iteration: 22170, Last Step Duration: 0.7995662689208984, Loss: 3.0627641677856445
Iteration: 22180, Last Step Duration: 0.8100674152374268, Loss: 3.0434889793395996
Iteration: 22190, Last Step Duration: 0.8596076965332031, Loss: 3.0666728019714355
Model Saved To: data\VanillaLSTM\model
Iteration: 22200, Last Step Duration: 0.7835545539855957, Loss: 3.0666537284851074
Iteration: 22210, Last Step Duration: 0.7975656986236572, Loss: 3.012580394744873
Iteration: 22220, Last Step Duration: 0.8065712451934814, Loss: 3.055081844329834
Iteration: 22230, Last Step Duration: 0.8028762340545654, Loss: 3.0373106002807617
Iteration: 22240, Last Step Duration: 0.8179619312286

Iteration: 23090, Last Step Duration: 0.8516037464141846, Loss: 3.020260810852051
Model Saved To: data\VanillaLSTM\model
Iteration: 23100, Last Step Duration: 0.8199150562286377, Loss: 3.031907558441162
Iteration: 23110, Last Step Duration: 0.8015503883361816, Loss: 3.093705892562866
Iteration: 23120, Last Step Duration: 0.7899744510650635, Loss: 3.07439923286438
Iteration: 23130, Last Step Duration: 0.8268358707427979, Loss: 3.0357584953308105
Iteration: 23140, Last Step Duration: 0.7906641960144043, Loss: 3.0743956565856934
Iteration: 23150, Last Step Duration: 0.7877297401428223, Loss: 3.020266532897949
Iteration: 23160, Last Step Duration: 0.8265864849090576, Loss: 3.0512146949768066
Iteration: 23170, Last Step Duration: 0.8045694828033447, Loss: 3.0434494018554688
Iteration: 23180, Last Step Duration: 0.8245711326599121, Loss: 3.0550713539123535
Iteration: 23190, Last Step Duration: 0.8085737228393555, Loss: 3.0975635051727295
Model Saved To: data\VanillaLSTM\model
Iteration: 2320